In [1]:
# Starting containers

!docker run --rm -p 3333:3333 inemo/isanlp
!docker run --rm --shm-size=1024m -ti -p 3334:9999 inemo/syntaxnet_rus server 0.0.0.0 9999
!docker run --rm -p 3335:3333 inemo/isanlp_srl_framebank

/bin/sh: 1: docker: not found
/bin/sh: 1: docker: not found
/bin/sh: 1: docker: not found


In [ ]:
!pip install grpcio
!pip install git+https://github.com/IINemo/isanlp
!pip install git+https://github.com/IINemo/isanlp_srl_framebank

In [7]:
%load_ext autoreload
%autoreload 2
    
import sys
sys.path.append('../src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import logging
logger = logging.getLogger('isanlp_srl_framebank')

import sys
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

logger.setLevel(logging.DEBUG)

In [9]:
from isanlp_srl_framebank.pipeline_default import PipelineDefault

ppl = PipelineDefault(address_morph=('192.168.1.69', 3333),
                      address_syntax=('192.168.1.69', 3334),
                      address_srl=('192.168.1.69', 3335))

res = ppl('Мы поехали на дачу.')
res

{'text': 'Мы поехали на дачу.',
 'tokens': [<isanlp.annotation.Token at 0x7fedb00a9d50>,
 'sentences': [<isanlp.annotation.Sentence at 0x7fedb00a7150>],
 'mystem_postag': [['SPRO,мн,1-л=им',
   'V,сов,нп=прош,мн,изъяв',
   'PR=',
   'S,жен,неод=вин,ед',
   '']],
 'lemma': [['мы', 'поехать', 'на', 'дача', '.']],
 'syntax_dep_tree': [[<isanlp.annotation.WordSynt at 0x7fedb00b4410>,
 'morph': [[{'fPOS': 'PRON', 'Number': 'Plur', 'Person': '1', 'Case': 'Nom'},
   {'fPOS': 'VERB',
    'Aspect': 'Perf',
    'Valency': 'INTR',
    'Tense': 'Past',
    'Number': 'Plur',
    'VerbForm': 'Fin'},
   {'fPOS': 'ADP'},
   {'fPOS': 'NOUN',
    'Gender': 'Fem',
    'Animacy': 'Inan',
    'Case': 'Acc',
    'Number': 'Sing'},
   {}]],
 'postag': [['PRON', 'VERB', 'ADP', 'NOUN', '']],
 'srl': [[<isanlp.annotation.Event at 0x7fedb00ba2d0>]]}

In [10]:
def print_roles(lemma, role_annot):
    for sent_num, ann_sent in enumerate(role_annot):
        for event in ann_sent:
            print('=====Pred: {}'.format(lemma[sent_num][event.pred[0]]))
            for arg in event.args:
                print('Arg({}): {}'.format(arg.tag, lemma[sent_num][arg.begin]))

print_roles(res['lemma'], res['srl'])

=====Pred: поехать
Arg(субъект перемещения): мы
Arg(конечная точка): дача
